In [1]:
import datetime
from pytz import timezone
print "Last run @%s" % (datetime.datetime.now(timezone('US/Pacific')))

Last run @2016-08-10 14:54:12.688600-07:00


In [2]:
import pandas as pd
print 'pandas: ',pd.__version__

pandas:  0.18.1


In [4]:
# Read Orders
orders = pd.read_csv('data/NW/NW-Orders.csv')
orders.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipCountry
0,10248,VINET,5,7/2/96,France
1,10249,TOMSP,6,7/3/96,Germany
2,10250,HANAR,4,7/6/96,Brazil
3,10251,VICTE,3,7/6/96,France
4,10252,SUPRD,4,7/7/96,Belgium


In [6]:
order_details = pd.read_csv('data/NW/NW-Order-Details.csv')
order_details.head()

,OrderID,ProductId,UnitPrice,Qty,Discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


In [8]:
products = pd.read_csv('data/NW/NW-Products.csv')
products.head()

,ProductID,ProductName,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,18.00,39,0,10,0
1,2,Chang,19.00,17,40,25,0
2,3,Aniseed Syrup,10.00,13,70,25,0
3,4,Chef Antons Cajun Seasoning,22.00,53,0,0,0
4,5,Chef Antons Gumbo Mix,21.35,0,0,0,1


In [9]:
orders.count()

OrderID        830
CustomerID     830
EmployeeID     830
OrderDate      830
ShipCountry    830
dtype: int64

In [10]:
order_details.count()

OrderID      2155
ProductId    2155
UnitPrice    2155
Qty          2155
Discount     2155
dtype: int64

In [11]:
products.count()

ProductID       77
ProductName     77
UnitPrice       77
UnitsInStock    77
UnitsOnOrder    77
ReorderLevel    77
Discontinued    77
dtype: int64

In [17]:
# Questions
# 1. How many orders were placed by each customer? 
# 2. How many orders were placed by each country ?
# 3. How many orders by month/year ?
# 4. Total Sales for each customer by year
# 5. Average order by customer by year
# These are questions based on customer and sales reports
# Similar questions can be asked about products as well

In [23]:
orders.dtypes

OrderID         int64
CustomerID     object
EmployeeID      int64
OrderDate      object
ShipCountry    object
dtype: object

In [22]:
# 1. How many orders were placed by each customer? 
orders.groupby("CustomerID").size().sort_values(ascending=False).to_frame().head()

,0
CustomerID,
SAVEA,31
ERNSH,30
QUICK,28
HUNGO,19
FOLKO,19


In [25]:
# 2. How many orders were placed by each country ?
orders.groupby("ShipCountry").size().sort_values(ascending=False).to_frame().head()

,0
ShipCountry,
USA,122
Germany,122
Brazil,82
France,77
UK,56


In [21]:
# For the next set of questions, let us transform the data
# 1. Add OrderTotal column to the Orders DataFrame
# 1.1. Add Line total to order details
# 1.2. Aggregate total by order id
# 1.3. Join order details & orders to add the order total
# 1.4. Check if there are any null columns
# 2. Add a date column
# 3. Add month and year

In [28]:
# 1.1. Add Line total to order details
order_details['OrderTotal'] = order_details['UnitPrice'] * order_details['Qty'] * \
                                       (1.0 - order_details['Discount'])
order_details.head()

,OrderID,ProductId,UnitPrice,Qty,Discount,OrderTotal
0,10248,11,14.0,12,0.0,168.0
1,10248,42,9.8,10,0.0,98.0
2,10248,72,34.8,5,0.0,174.0
3,10249,14,18.6,9,0.0,167.4
4,10249,51,42.4,40,0.0,1696.0


In [66]:
# 1.2. Aggregate total by order id
order_tot = order_details.groupby('OrderID').sum()["OrderTotal"].to_frame()
order_tot.head()

,OrderTotal
OrderID,
10248,440.00
10249,1863.40
10250,1552.60
10251,654.06
10252,3597.90


The OrderId is a label, not a column. WHich is no problem as wehave the reset_index() !

In [67]:
order_tot = order_tot.reset_index()
order_tot.head()

,OrderID,OrderTotal
0,10248,440.00
1,10249,1863.40
2,10250,1552.60
3,10251,654.06
4,10252,3597.90


In [63]:
orders_1 = orders.merge(order_tot)
orders_1.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipCountry,OrderTotal
0,10248,VINET,5,7/2/96,France,440.00
1,10249,TOMSP,6,7/3/96,Germany,1863.40
2,10250,HANAR,4,7/6/96,Brazil,1552.60
3,10251,VICTE,3,7/6/96,France,654.06
4,10252,SUPRD,4,7/7/96,Belgium,3597.90


In [64]:
# Test Correctness
orders_1[orders_1['OrderID'] == 11011]
#+-------+----------+---------+-----------+------------------+
#|OrderID|CustomerID|OrderDate|ShipCountry|             Total|
#+-------+----------+---------+-----------+------------------+
#|  10643|     ALFKI|  8/23/97|    Germany|             814.5|
#|  10952|     ALFKI|  3/14/98|    Germany|471.19999665021896|
#|  10692|     ALFKI|  10/1/97|    Germany|             878.0|
#|  10702|     ALFKI| 10/11/97|    Germany|             330.0|
#|  10835|     ALFKI|  1/13/98|    Germany|  845.799999922514|
#|  11011|     ALFKI|   4/7/98|    Germany| 933.4999996051192|
#|  10759|     ANATR| 11/26/97|     Mexico|             320.0|
#|  10308|     ANATR|  9/16/96|     Mexico| 88.79999923706055|
#|  10625|     ANATR|   8/6/97|     Mexico|            479.75|
#|  10926|     ANATR|   3/2/98|     Mexico| 514.4000015258789|
#|  10856|     ANTON|  1/26/98|     Mexico|             660.0|
#|  10682|     ANTON|  9/23/97|     Mexico|             375.5|
#|  10365|     ANTON| 11/25/96|     Mexico| 403.1999816894531|
#|  10677|     ANTON|  9/20/97|     Mexico| 813.3649631685021|
#|  10507|     ANTON|  4/13/97|     Mexico| 749.0624947473407|
#|  10535|     ANTON|  5/11/97|     Mexico|1940.8499967865646|
#|  10573|     ANTON|  6/17/97|     Mexico|            2082.0|
#|  10383|     AROUT| 12/14/96|         UK|             899.0|
#|  10355|     AROUT| 11/13/96|         UK|             480.0|
#|  10768|     AROUT|  12/6/97|         UK|            1477.0|
#+-------+----------+---------+-----------+------------------+
#only showing top 20 rows'''

,OrderID,CustomerID,EmployeeID,OrderDate,ShipCountry,OrderTotal
763,11011,ALFKI,3,4/7/98,Germany,933.5


In [65]:
# 1.4. Check if there are any null columns
orders_1[pd.isnull(orders_1['OrderTotal'])]

,OrderID,CustomerID,EmployeeID,OrderDate,ShipCountry,OrderTotal


In [70]:
orders_1["OrdDate"] = pd.to_datetime(orders_1["OrderDate"])
orders_1.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipCountry,OrderTotal,OrdDate
0,10248,VINET,5,7/2/96,France,440.00,1996-07-02
1,10249,TOMSP,6,7/3/96,Germany,1863.40,1996-07-03
2,10250,HANAR,4,7/6/96,Brazil,1552.60,1996-07-06
3,10251,VICTE,3,7/6/96,France,654.06,1996-07-06
4,10252,SUPRD,4,7/7/96,Belgium,3597.90,1996-07-07


In [73]:
# 3. Add month and year
orders_1['Month'] = orders_1["OrdDate"].dt.month
orders_1['Year'] = orders_1["OrdDate"].dt.year
orders_1.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipCountry,OrderTotal,OrdDate,Month,Year
0,10248,VINET,5,7/2/96,France,440.00,1996-07-02,7,1996
1,10249,TOMSP,6,7/3/96,Germany,1863.40,1996-07-03,7,1996
2,10250,HANAR,4,7/6/96,Brazil,1552.60,1996-07-06,7,1996
3,10251,VICTE,3,7/6/96,France,654.06,1996-07-06,7,1996
4,10252,SUPRD,4,7/7/96,Belgium,3597.90,1996-07-07,7,1996


In [81]:
# 3. How many orders by month/year ?
orders_1.groupby(["Year","Month"]).size().reset_index().head(10)

,Year,Month,0
0,1996,7,25
1,1996,8,23
2,1996,9,24
3,1996,10,25
4,1996,11,25
5,1996,12,33
6,1997,1,30
7,1997,2,29
8,1997,3,33
9,1997,4,31


In [92]:
# 3.a. Total value of orders by month/year ?
%time
orders_1.groupby(["Year","Month"]).sum().reset_index()[['Year','Month','OrderTotal']].head(10)
# 2015-12-17 21:31:20.033784 Elapsed : 17.007098
# 8/10/16
# CPU times: user 4 µs, sys: 5 µs, total: 9 µs
# Wall time: 11.9 µs

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


,Year,Month,OrderTotal
0,1996,7,30741.8950
1,1996,8,22726.8750
2,1996,9,27691.4000
3,1996,10,38380.1250
4,1996,11,45694.4450
5,1996,12,52494.3300
6,1997,1,51612.9700
7,1997,2,38483.6350
8,1997,3,40918.8200
9,1997,4,57116.7125


In [94]:
# 4. Total Sales for each customer by year
%time
orders_1.groupby(["CustomerID","Year"]).sum().reset_index()[['CustomerID','Year','OrderTotal']].head(10)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


,CustomerID,Year,OrderTotal
0,ALFKI,1997,2022.5000
1,ALFKI,1998,2250.5000
2,ANATR,1996,88.8000
3,ANATR,1997,799.7500
4,ANATR,1998,514.4000
5,ANTON,1996,403.2000
6,ANTON,1997,5960.7775
7,ANTON,1998,660.0000
8,AROUT,1996,1379.0000
9,AROUT,1997,6406.9000


In [95]:
# 5. Average order by customer by year
%time
orders_1.groupby(["CustomerID","Year"]).mean().reset_index()[['CustomerID','Year','OrderTotal']].head(10)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


,CustomerID,Year,OrderTotal
0,ALFKI,1997,674.166667
1,ALFKI,1998,750.166667
2,ANATR,1996,88.800000
3,ANATR,1997,399.875000
4,ANATR,1998,514.400000
5,ANTON,1996,403.200000
6,ANTON,1997,1192.155500
7,ANTON,1998,660.000000
8,AROUT,1996,689.500000
9,AROUT,1997,915.271429


In [96]:
# 6. Average order by customer
orders_1.groupby(["CustomerID"]).mean().reset_index()[['CustomerID','OrderTotal']].head(10)

,CustomerID,OrderTotal
0,ALFKI,712.166667
1,ANATR,350.737500
2,ANTON,1003.425357
3,AROUT,1030.050000
4,BERGS,1384.865417
5,BLAUS,462.828571
6,BLONP,1684.916364
7,BOLID,1410.950000
8,BONAP,1291.956029
9,BOTTM,1485.828571


### _That's All, Folks !_